In [27]:
import re
import math
import numpy as np
import pandas as pd
from numpy import linalg
from numpy import random
from xml.etree import ElementTree
from nltk.stem.porter import PorterStemmer;

xml = open("data/test_data.xml", 'r').read()
root = ElementTree.fromstring(re.sub(r"(<\?xml[^>]+\?>)", r"\1<root>", xml) + "</root>")
count = re.findall('<\/REUTERS>', xml)

with open("stopwords.txt") as f:    
    stopwords = f.read();

regex = re.compile(r"\b([a-z])+\b");
stopwords = set(stopwords.split());
data = {}
all_words = set()
f.close()

def filter_reci(reci, stopwords):
    t = reci.replace("\"", "").replace("/", " ").replace("-", " ").replace("\n", "").replace(",", "").replace("(", "").replace(")", "").replace(".", "").replace(r"[ ]+", "").lower()
    t = [w for w in t.split() if regex.match(w) is not None]
    t = [w for w in t if w not in stopwords]
    return t

f = open('atributi.csv', 'w')
f.write('TOPICS;PLACE;PEOPLE;ORGS;EXCHANGES' + '\n')

def filter_atributa(arg):
        tmp = []
        for content in arg:
            tmp.append(content.text)
        if(len(tmp) == 0):
            t = 'None'
        else:
            t = tmp[0]
        tmp.clear()
        return t

porterStemmer = PorterStemmer()
    
for i in range(0, len(count)):
    
    data[i] = []
    
    #prepoznajemo reci
    reci = root[i][7][2].text

    #ucitavanje znacajnih atributa koje cemo koristiti u slucaju ispisa
    topic = filter_atributa(root[i][1].getchildren())
    place = filter_atributa(root[i][2].getchildren())
    people = filter_atributa(root[i][3].getchildren())
    orgs = filter_atributa(root[i][4].getchildren())
    exchanges = filter_atributa(root[i][5].getchildren())
    
    f.write(topic + ";" + place + ";" + people +";" + orgs +";" + exchanges +'\n')
    
    #filtriramo reci
    reci = filter_reci(reci, stopwords)
    reci = [porterStemmer.stem(w) for w in reci];
    #dodajemo reci u razlicite skupove da bismo mogli izracunati bitnost reci
    data[i].append(reci)
    all_words.update(reci)

f.close()

def tf(word,status):
    cnt = sum (1 for w in status if w == word)
    return cnt / float(len(status))

def idf(word,statuses):
    cnt = sum (1 for status in statuses if word in status)
    return math.log(len(count) / (1 + cnt))

words = []
import operator
for f in range(0, len(data)):
    tekstovi = data[f]
    for tekst in tekstovi:
        if len(tekst) == 0:
            continue
        sorted_number_word = []
        number_of_words = {}
        
        for word in all_words:
            number_of_words.update({word:tf(word,tekst) * idf(word,tekstovi)})
            
        sorted_number_word = sorted(number_of_words.items(), key=operator.itemgetter(1))
        sub_words = sorted_number_word[-5:]
        for i in range(0, 5):
            words.append(sub_words[i])
w = []
title = ""
for i in range(0, len(words)):
    if words[i][0] not in w:
        w.append(words[i][0])
        title += words[i][0] + ";"
title = title[:-1]
title

#Formiramo fajl za klasterovanje
f = open('data_parsed.csv', 'w')
f.write(title + '\n')

for i in range(0, len(data)):
    body = ""
    row = np.array([])
    row.resize(len(w))
    for j in range(0,5):
        k = (i*5) + j
        l = w.index(words[k][0])
        row[l] = words[k][1]
    for m in range(0, len(row)):
        body += str(row[m])+";"
    body = body[:-1]
    f.write(body + "\n")
f.close()

data = pd.read_csv('data_parsed.csv', sep=';').as_matrix()

#inicijalizacija matrice pripadnosti instance svakom klasteru
broj_instanci = data.shape[0]
broj_atributa = data.shape[1]

#setovati broj klastera
broj_klastera = 2

p = np.zeros((broj_instanci,broj_klastera))
pp = np.zeros((broj_instanci,broj_klastera))

# inicijalizacija random gausovih parametra
cpl = []
for i in range(0, broj_klastera):
    cpl.append(np.random.uniform(0, 1))

cplp = cpl.copy()

cpm = []
for i in range(0, broj_klastera):
    tmp = np.zeros(broj_atributa)
    for j in range(0, broj_atributa):
        minv = data[:,j].min()
        maxv = data[:,j].max()
        tmp[j] = np.random.uniform(minv, maxv)
    cpm.append(tmp)
cpm = np.array(cpm)
    
cov = np.eye(broj_atributa)
cps = np.random.multivariate_normal(cpm[0], cov, (broj_atributa,broj_atributa))    
    
def verovatnoca(x, cpl, cpm, cps, broj_atributa):
    a = cpl*(1/((np.power((2*np.pi),broj_atributa/2)) * np.sqrt((np.linalg.norm(cps)))))
    b = np.exp((-np.dot(np.dot((x-cpm).T, cps),(x-cpm))/2))
    return a*b

def pripada_klasteru(p):
    ci = []
    for i in range(0, broj_instanci):
        a = np.array(p[i,:])
        i = np.where(a == a.max())
        ci.append(i[0][0])
    return ci

def normalize(value, suma):
    normalized = value/suma
    return normalized

def E_step(cpl, cpm, cps):
    p = np.zeros((broj_instanci, broj_klastera))
    for i in range(0, broj_instanci):
        for j in range(0, broj_klastera):
            p[i,j] = verovatnoca(data[i], cpl[j], cpm[j], cps[j], broj_atributa)

    for i in range(0, broj_instanci):
        for j in range(0, broj_klastera):
            value = p[i,j]
            pp[i,j] = normalize(value, np.sum(p[i,:]))
    return pp;

def M_step(pp):
    #azuriranje cpl
    cplp = cpl.copy()
    for i in range(0, broj_klastera):
        cpl[i] = np.sum(pp[:,i])/broj_instanci

    #azuriranje cpm
    for i in range(0, broj_klastera):
        for j in range(0, broj_atributa):
            s = 0
            for k in range(0, broj_instanci):
                s += data[k,i] * pp[k, i]
            sc = np.sum(pp[:,i])
            cpm[i,j] = s/sc

    #azuriranje cps
    for i in range(0, broj_klastera):
        for j in range(0, broj_atributa):
            for k in range(0, broj_atributa):
                s = 0
                for l in range(0, broj_instanci):
                    pr = pp[l,i]
                    dr = data[l,:] - cpm[i]
                    tr = (data[l,:] - cpm[i]).reshape(broj_atributa, 1)
                    s += pr * np.dot(dr, tr)
                cps[i][j][k] = s/np.sum(pp[:,i])
    return cpl, cpm, cps, cplp


def konvergira(cpl, cplp, eps, iteracija, broj_atributa, br_iteracija):
    if iteracija == 1:
        return False
    
    if iteracija == br_iteracija:
        return True
    
    for i in range(0, broj_atributa):
        if np.abs(cpl[i] - cplp[i]) < eps:
            return True
        else:
            return False
eps = 0.01
iteracija = 1
br_iteracija = np.random.randint(1,5)

while(konvergira(cpl,cplp, eps, iteracija, broj_atributa, br_iteracija) != True):
    pp = E_step(cpl, cpm, cps)
    cpl, cpm, cps, cplp = M_step(pp)
    iteracija = iteracija + 1

atributi = pd.read_csv("atributi.csv", sep=";").as_matrix()

pripada = np.array(pripada_klasteru(pp))
cluster = []
for i in range(0, broj_klastera):
    tmp = []
    for j in range(0, len(pripada)):
        if pripada[j] == i:
            tmp.append(atributi[j,:])
    cluster.append(tmp)    

print('Broj potrebnih iteracija je:', iteracija-1, '\n')
for i in range(0, broj_klastera):
    print(i+1 , 'klasteru pripadaju sledece instance:')
    for j in range(0, len(cluster[i])):
        print(cluster[i][j])
    print('\n')

Broj potrebnih iteracija je: 1 

1 klasteru pripadaju sledece instance:
['None' 'usa' 'None' 'None' 'None']
['grain' 'usa' 'None' 'None' 'None']


2 klasteru pripadaju sledece instance:
['cocoa' 'el-salvador' 'None' 'None' 'None']
['None' 'usa' 'None' 'None' 'None']
['None' 'usa' 'None' 'None' 'None']
['veg-oil' 'argentina' 'None' 'None' 'None']
['None' 'usa' 'None' 'None' 'None']
['None' 'usa' 'None' 'None' 'None']
['earn' 'usa' 'None' 'None' 'None']
['acq' 'usa' 'None' 'None' 'None']
['earn' 'usa' 'None' 'None' 'None']
['None' 'usa' 'None' 'None' 'None']
['None' 'usa' 'None' 'None' 'None']
['trade' 'uk' 'thatcher' 'None' 'None']
['None' 'usa' 'None' 'None' 'None']
['None' 'usa' 'None' 'None' 'None']
['earn' 'usa' 'None' 'None' 'None']


